<h1>Juice Problem Project  *Link to Report at Bottom of the code*</h1>

In this notebook we use <code>pulp</code> to solve the Juice problem from the text.

In [ ]:
!pip install pulp
import itertools
import numpy as np
import pulp

     |████████████████████████████████| 13.6MB 154kB/s 
  Created wheel for pulp: filename=PuLP-1.6.10-cp36-none-any.whl size=12269903 sha256=99cc829275deb14a645bfa54f0a5dc1751b144f4d54afbd24b5d65ad7604a530
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


In [ ]:
#Cost to transport juice
CostT= [65,65,65,65,65,65,65,65,65,65,65,65]

#Cost to manufacture 1 ton of each product
CostM= np.array([
    [150,150],
    [175,175]
])

#Cost of storage at each plant per season a.k.a. per year
CostS= [500,500]

print(CostT)
print(CostM)
print(CostS)

[65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65]
[[150 150]
 [175 175]]
[500, 500]


We first create LpProblem in pulp.

In [ ]:
JuiceLP = pulp.LpProblem("Juice Problem",pulp.LpMinimize)

We next create the decision variables.  These come in Four types: $TS_{i,j,k}$ is the amount of product shipped $j$ from Plant $k$ to stores during month $i$. while $TS_{i,k,m}$ is the amount of juice transffered into plant $k$ from plant $m$ in month $i$. Also we have $TO_{i,k,m}$ is the amount of juice transffered out of plant $k$ and into plant $m$ in month $i$. Finally $EI_{i,k}$ which is the amount of grape juice at plant $k$ at the end of the month $i$.

---



In [ ]:
Plants = range(2)
Months = range(12)
Products = range(2)
Season = range (1)
AllowedPairs = list(itertools.product(Plants,Plants))
AllowedPairs.remove((0,0))
AllowedPairs.remove((1,1))
print(AllowedPairs)

TS = pulp.LpVariable.dicts("TS",(Months,Products,Plants),0,None,pulp.LpContinuous)
TI = pulp.LpVariable.dicts("TI",(Months, Plants),0,None,pulp.LpContinuous)
TO = pulp.LpVariable.dicts("TO",(Months,Plants),0,None,pulp.LpContinuous)
EI = pulp.LpVariable.dicts("EI",(Months,Plants),0,None,pulp.LpContinuous)

#Amount delivered to plant 0 at the beginning of each month
P1= [15*(1+np.sin(np.pi/6*(i+1))) for i in range(12)]

#Amount delivered to plant 0 at the beginning of each month
P2= [50*(1+np.sin(np.pi/6*(i+1))) for i in range(12)]

print(P1)
print(P2)

#Amount of juice required to create 1 ton of juice of jam at each plant
Production= np.array([[3,2],
                      [4,2]])

#Max combined production for the year
MaxCombined=[150,150]

#Max amount of product allowed to be shipped per month
MaxShipment=[50,50]

#Amount of product required to be produced by the plants each month
Demand=[10,15]

#Amount of juice required at the end of each year in each plant.
RequiredJuice=[70,70]

#P1=[23,28,30,28,23,15,8,2,0,2,8,15]
#P2=[75,93,100,93,75,50,25,7,0,7,25,50]

[(0, 1), (1, 0)]
[22.5, 27.99038105676658, 30.0, 27.990381056766584, 22.500000000000007, 15.000000000000004, 7.5000000000000036, 2.0096189432334244, 0.0, 2.0096189432334146, 7.499999999999993, 14.999999999999996]
[75.0, 93.30127018922192, 100.0, 93.30127018922194, 75.00000000000003, 50.000000000000014, 25.00000000000001, 6.698729810778081, 0.0, 6.6987298107780475, 24.99999999999998, 49.999999999999986]


Now we add the objective function.

In [ ]:
JuiceLP += sum( EI[11][k]*CostS[k]for k in Plants) + sum(TS[i][j][k]*CostM[j][k] for i in Months for k in Plants for j in Products) + sum(TI[i][k]*2*CostT[i] for i in Months for k in Plants) + sum(TO[i][k]*-1*CostT[i] for i in Months for k in Plants)

In [ ]:

#At the end of the season the Juice in storage in each plant should be 70 tons
for k in Plants:
  JuiceLP += (EI[11][k]) >= RequiredJuice[k]
#The amount shipped from Plant 0 is equal to the amount recieved at Plant 1
for i in Months:
    JuiceLP+= (TO[i][0]) == (TI[i][1])
#The amount shipped from Plant 1 is equal to the amount recieved at Plant 0
for i in Months:
    JuiceLP+= (TO[i][1]) == (TI[i][0])

#The total monthly production should meet the demands
for i in Months:
  for j in Products:
    JuiceLP+= sum(TS[i][j][k] for k in Plants)==Demand[j]


#Limit on how much juice can be shipped between plants
for i in Months:
   for k in Plants:
    JuiceLP+= (TI[i][k])<=MaxShipment[k]

#Limit on how much jam and juice concentrate each plant can produce anually
for k in Plants:
  JuiceLP+= sum(TS[i][j][k] for i in Months for j in Products) <= MaxCombined[k]

#This shows how much juice was being stored at the end of month 0 at plant 0
JuiceLP+=EI[0][0]==70+TI[0][0]-TO[0][0]-sum(TS[0][j][0]*Production[0][j] for j in Products)+P1[0]

#This shows how much juice was being stored at the end of month 0 at plant 1
  JuiceLP+=EI[0][1]==70+TI[0][1]-TO[0][1]-sum(TS[0][j][1]*Production[1][j] for j in Products)+P2[0]


IndentationError: ignored

In [ ]:
#This shows how much juice was being stored at the end of months 1-11 at plant 1

Months2 = [1,2,3,4,5,6,7,8,9,10,11]
for i in Months2:
  JuiceLP+=EI[i][1]==EI[i-1][1]+TI[i][1]-TO[i][1]-sum(TS[i][j][1]*Production[1][j] for j in Products)+P2[i]

In [ ]:
#This shows how much juice was being stored at the end of months 1-11 at plant 0

for i in Months2:
  JuiceLP+=EI[i][0]==EI[i-1][0]+TI[i][0]-TO[i][0]-sum(TS[i][j][0]*Production[0][j] for j in Products)+P1[i]
  print(JuiceLP)

Our LP is complete and we are ready to solve.

In [ ]:
JuiceLP.solve()
print("Status:", pulp.LpStatus[JuiceLP.status])

In [ ]:
pulp.value(JuiceLP.objective)

In [ ]:
for v in JuiceLP.variables():
    print(v.name, "=", v.varValue)



Finally we can print the solution as a pair of arrays.

In [ ]:
print("Total cost is", pulp.value(JuiceLP.objective))

SolutionArray=np.array([[[TS[i][j][k].varValue for i in Months] for j in Products] for k in Plants])
SolutionArray2=np.asarray([[TI[i][k].varValue for i in Months] for k in Plants])
SolutionArray3=np.asarray([[TO[i][k].varValue for i in Months] for k in Plants])
SolutionArray4=np.asarray([[EI[i][k].varValue for i in Months] for k in Plants])

print("TS Array:")
print(SolutionArray)
print("TI Array:")
print(SolutionArray2)
print("TO Array:")
print(SolutionArray3)
print("EI Array:")
print(SolutionArray4)

<h1>  *Link to Report*</h1>

https://docs.google.com/document/d/1kfajfpNvkDv4ZxsTPJmlPru3pozDDTinVkMWDpFAVW0/edit